In [1]:
import pandas as pd
import json
import requests

In [3]:
def generate_data (
      start = 1,
      end = 2,
      api_key = ''):
      base_url = f"https://history.openweathermap.org/data/2.5/history/city?q=Bogor,ID&type=hour&start=1656651600&cnt=1662008400&appid="
      response = requests.get(base_url)
      x = response.json()
      return base_url, x

In [ ]:
from : 1656651600
to : 1662008400

In [4]:
generate_data(start=1656651600, end=1)

('https://history.openweathermap.org/data/2.5/history/city?q=Bogor,ID&type=hour&start=1656651600&cnt=1662008400&appid=ca6550c30608f0045c58b837f8210119',
 {'message': 'Count: 169',
  'cod': '200',
  'city_id': 1648473,
  'calctime': 0.121668053,
  'cnt': 169,
  'list': [{'dt': 1656651600,
    'main': {'temp': 302.16,
     'feels_like': 303.61,
     'pressure': 1009,
     'humidity': 56,
     'temp_min': 302.16,
     'temp_max': 302.16},
    'wind': {'speed': 1.55, 'deg': 19, 'gust': 1.84},
    'clouds': {'all': 73},
    'weather': [{'id': 803,
      'main': 'Clouds',
      'description': 'broken clouds',
      'icon': '04d'}]},
   {'dt': 1656655200,
    'main': {'temp': 302.67,
     'feels_like': 304.1,
     'pressure': 1008,
     'humidity': 54,
     'temp_min': 302.67,
     'temp_max': 302.67},
    'wind': {'speed': 1.96, 'deg': 33, 'gust': 2.59},
    'clouds': {'all': 72},
    'weather': [{'id': 803,
      'main': 'Clouds',
      'description': 'broken clouds',
      'icon': '04d'}]}

In [2]:
requests.get('https://api.openweathermap.org/data/2.5/weather?id=2172797&appid=58cea55961dd9dd67a0a958061d06a3b').json()

{'coord': {'lon': 145.7667, 'lat': -16.9167},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 297.89,
  'feels_like': 298.85,
  'temp_min': 297.63,
  'temp_max': 298.12,
  'pressure': 1008,
  'humidity': 93},
 'visibility': 10000,
 'wind': {'speed': 4.12, 'deg': 170},
 'clouds': {'all': 75},
 'dt': 1676995015,
 'sys': {'type': 1,
  'id': 9490,
  'country': 'AU',
  'sunrise': 1677010458,
  'sunset': 1677055626},
 'timezone': 36000,
 'id': 2172797,
 'name': 'Cairns',
 'cod': 200}

In [22]:
import requests, json
 
# Enter your API key here
api_key = "58cea55961dd9dd67a0a958061d06a3b"
 
# base_url variable to store url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
 
# Give city name
city_name = input("Enter city name : ")
 
# complete_url variable to store
# complete url address
complete_url = base_url + "appid=" + api_key + "&q=" + city_name
 
# get method of requests module
# return response object
response = requests.get(complete_url)
 
# json method of response object
# convert json format data into
# python format data
x = response.json()

In [23]:
x

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}